Import the data :

mongoimport --type csv -d energy_data -c renewables --headerline --drop MER_T10_01.csv
mongoimport --type csv -d energy_data -c biomass --headerline --drop biomass_energy_consump2.csv
mongoimport --type csv -d energy_data -c geothermal --headerline --drop geothermal_energy_consump2.csv
mongoimport --type csv -d energy_data -c hydropower --headerline --drop hydropower_energy_consump2.csv
mongoimport --type csv -d energy_data -c solar --headerline --drop solar_energy_consump2.csv
mongoimport --type csv -d energy_data -c wind --headerline --drop wind_energy_consump2.csv


In [1]:
from pymongo import MongoClient
from pprint import pprint
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from IPython.display import display

In [2]:
midwest_geojson = 'midwest.json'
gdf = gpd.read_file(midwest_geojson)

In [3]:

# Calculate the mean of the geometries to set the center manually
center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()

# Create a folium.Map object with the manually set center
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Display the map
m



C:\Users\hespr\AppData\Local\Temp\ipykernel_24120\2615464939.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf.geometry.centroid.y.mean()
C:\Users\hespr\AppData\Local\Temp\ipykernel_24120\2615464939.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf.geometry.centroid.x.mean()


In [12]:
table_10 = pd.read_csv('MER_T10_01.csv')
biomass_data = pd.read_csv('biomass_energy_consump2.csv')
geothermal_data = pd.read_csv('geothermal_energy_consump2.csv')
hydro_data = pd.read_csv('hydropower_energy_consump2.csv')
solar_data = pd.read_csv('solar_energy_consump2.csv')
wind_data = pd.read_csv('wind_energy_consump2.csv')


In [13]:
# table_10.head()
biomass_data.head()
# geothermal_data.head()
# hydro_data.head()
# solar_data.head()
# wind_data.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AK,"2,069","3,445","3,540","3,494","3,783","1,197","1,249","1,426","1,447",...,"2,298","3,660","6,494","7,530","9,181","7,802","8,231","7,668","8,687","8,503"
1,AL,"203,782","166,334","163,686","156,371","186,726","178,417","194,853","188,449","177,209",...,"194,992","213,553","203,628","192,270","194,485","198,072","196,226","193,284","187,599","196,574"
2,AR,"83,466","66,837","72,972","80,406","75,933","81,477","84,758","89,239","79,760",...,"103,550","106,124","105,604","95,088","93,707","93,283","94,536","92,332","74,385","72,036"
3,AZ,"13,376","10,389","9,319","9,573","9,704","25,335","25,421","29,420","36,719",...,"27,209","27,187","33,300","35,397","37,709","37,953","40,107","40,871","35,235","37,237"
4,CA,"164,105","164,137","171,500","205,815","228,444","226,206","220,116","225,983","230,785",...,"296,354","343,657","349,289","344,065","349,253","357,861","357,070","405,324","400,214","467,144"


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'energy_data', 'epa', 'fruits_db', 'gardenDB', 'local', 'marketing', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo['energy_data']
db_st = mongo['state_data']

In [5]:
# review the collections in thedatabase
print(db.list_collection_names())

['renewables']


In [6]:
# review a document in the collection
print(db.renewables.find_one())

{'_id': ObjectId('65b46a42db1d32e044e87536'), 'MSN': 'WDPRBUS', 'YYYYMM': 194913, 'Value': 1549.262, 'Column_Order': 1, 'Description': 'Wood Energy Production', 'Unit': 'Trillion Btu'}


In [7]:
# assign the collection to a variable
renewables = db['renewables']
state_prod_v_consum = db_st['state_prod_v_consum']

In [124]:
import folium
from ipywidgets import Dropdown
from IPython.display import display
import pandas as pd

# # Sample data - you need to replace this with your actual data
# data = {
#     'State': ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin'],
#     'Year': [2000, 2001, 2002, 2003, 2004, 2005, 2009, 2010, 2015, 2019, 2020, 2021],
#     'Biomass': [100, 120, 80, 150, 200, 90, 110, 130, 170, 190, 120, 150],
#     'Hydropower': [50, 60, 40, 70, 80, 45, 55, 65, 85, 95, 60, 75],
#     'Geothermal': [30, 35, 25, 40, 50, 30, 35, 40, 45, 50, 35, 40],
#     'Solar': [20, 25, 15, 30, 40, 20, 25, 30, 35, 40, 25, 30],
#     'Wind': [150, 180, 120, 200, 250, 130, 160, 180, 220, 240, 170, 200],
# }

# df = pd.DataFrame(data)

# biomass_data = pd.read_csv('biomass_energy_consump.csv')
# geothermal_data = pd.read_csv('geothermal_energy_consump.csv')
# hydro_data = pd.read_csv('hydropower_energy_consump.csv')
# solar_data = pd.read_csv('solar_energy_consump.csv')
# wind_data = pd.read_csv('wind_energy_consump.csv')

df = biomass_data.set_index('State')
df['Geothermal'] = geothermal_data.set_index('State')['Geothermal']
df['Hydropower'] = hydro_data.set_index('State')['Hydropower']
df['Solar'] = solar_data.set_index('State')['Solar']
df['Wind'] = wind_data.set_index('State')['Wind']

# Create a map centered around the Midwest
midwest_map = folium.Map(location=[41.8781, -87.6298], zoom_start=5)

# Dictionary to store markers
markers_dict = {}

# Function to add markers for a given year
def add_markers_for_year(selected_year):
    # Clear existing markers
    clear_markers()

    # Filter the DataFrame for the selected year
    selected_data = df[df['Year'] == selected_year]

    # Add markers for the selected year
    for _, row in selected_data.iterrows():
        state = row['State']
        biomass = row['Biomass']
        hydropower = row['Hydropower']
        geothermal = row['Geothermal']
        solar = row['Solar']
        wind = row['Wind']

        # Get the location for each state
        location = get_location(state)

        # Create a popup message with the data
        popup_text = f"""
        State: {state}
        Year: {selected_year}
        Biomass: {biomass}
        Hydropower: {hydropower}
        Geothermal: {geothermal}
        Solar: {solar}
        Wind: {wind}
        """

        # Add a marker for each state
        markers_dict[state] = folium.Marker(
            location=location,
            popup=popup_text,
            icon=folium.Icon(color='green'),
        )
        markers_dict[state].add_to(midwest_map)

# Function to clear existing markers
def clear_markers():
    for marker in markers_dict.values():
        marker.add_to(None)

# Function to get the location for a given state
def get_location(state):
    # Replace with the actual latitude and longitude for each state
    if state == 'Illinois':
        return [39.799999, -89.650002]
    elif state == 'Indiana':
        return [39.791000, -86.148003]
    elif state == 'Iowa':
        return [41.619549, -93.598022]
    elif state == 'Kansas':
        return [39.056198, -95.695312]
    elif state == 'Michigan':
        return [42.732536, -84.555534]
    elif state == 'Minnesota':
        return [44.949642, -93.093124]
    elif state == 'Missouri':
        return [38.579575, -92.174782]
    elif state == 'Nebraska':
        return [40.806862, -96.681679]
    elif state == 'North Dakota':
        return [46.825905, -100.778275]
    elif state == 'Ohio':
        return [39.983334, -82.983330]
    elif state == 'South Dakota':
        return [44.3675, -100.3352]
    elif state == 'Wisconsin':
        return [43.073051, -89.401230]

# Create a dropdown widget for selecting years
year_dropdown = Dropdown(options=df['Year'].unique(), value=df['Year'].min(), description='Year:')
display(year_dropdown)

# Function to update the map based on the selected year
def update_map(change):
    selected_year = change['new']
    add_markers_for_year(selected_year)

# Connect the function to the dropdown's value change event
year_dropdown.observe(update_map, names='value')

# Display the map
midwest_map


KeyError: 'Geothermal'

In [15]:
import folium
from ipywidgets import Dropdown
from IPython.display import display
import pandas as pd

# Function to load data from CSV file
def load_energy_data(file_path, energy_type):
    data = pd.read_csv(file_path)
    data.set_index('State', inplace=True)
    data.rename(columns=lambda x: f"{energy_type}_{x}", inplace=True)
    return data

# Load data from each CSV file
biomass_data = load_energy_data('biomass_energy_consump2.csv', 'Biomass')
geothermal_data = load_energy_data('geothermal_energy_consump2.csv', 'Geothermal')
hydro_data = load_energy_data('hydropower_energy_consump2.csv', 'Hydropower')
solar_data = load_energy_data('solar_energy_consump2.csv', 'Solar')
wind_data = load_energy_data('wind_energy_consump2.csv', 'Wind')

# Consolidated  the data into a single dataframe
df = pd.concat([biomass_data, geothermal_data, hydro_data, solar_data, wind_data], axis=1)


state_mapping = {
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'NE': 'Nebraska',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'SD': 'South Dakota',
    'WI': 'Wisconsin'
}

# Create a map centered around the Midwest
midwest_map = folium.Map(location=[41.8781, -87.6298], zoom_start=5)

# Dictionary to store markers
markers_dict = {}
selected_year = 2000
def add_markers_for_year_and_energy(selected_year, energy_type):
    # Clear existing markers
    clear_markers()
    print(selected_year)
    print(energy_type)
    
    def handle_click(**kwargs):
        lat, lon = kwargs['location']
        state = get_state_from_location(lat, lon)
        selected_year = year_dropdown.value
        selected_energy = energy_dropdown.value
        display_state_data(state, selected_year, selected_energy)

    # Function to get the state from the clicked location
    def get_state_from_location(lat, lon):
    # Replace this with the logic to determine the state based on latitude and longitude
    # For simplicity, let's assume you have a function that returns the state name
        return "Illinois"  # Replace with the actual logic

    # Function to display the state data
    def display_state_data(state, selected_year, selected_energy):
        clear_markers()  # Clear existing markers

        # Get the location for each state
        location = get_location(full_state_name)

        # Create a popup message with the data for the selected energy type
        popup_text = f"""
        State: {full_state_name}
        Year: {selected_year}
        {energy_type}: {row[energy_type + '_' + selected_year]}
        """
        # Add a marker for each state
        markers_dict[state] = folium.Marker(
            location=location,
            popup=popup_text,
            icon=folium.Icon(color='green'),
        )
        markers_dict[state].add_to(midwest_map)

# Function to clear existing markers
def clear_markers():
    for marker in markers_dict.values():
        marker.add_to(None)

# Function to get the location for a given state
def get_location(state):
    # Replace with the actual latitude and longitude for each state
    if state == 'Illinois':
        return [39.799999, -89.650002]
    elif state == 'Indiana':
        return [39.791000, -86.148003]
    elif state == 'Iowa':
        return [41.619549, -93.598022]
    elif state == 'Kansas':
        return [39.056198, -95.695312]
    elif state == 'Michigan':
        return [42.732536, -84.555534]
    elif state == 'Minnesota':
        return [44.949642, -93.093124]
    elif state == 'Missouri':
        return [38.579575, -92.174782]
    elif state == 'Nebraska':
        return [40.806862, -96.681679]
    elif state == 'North Dakota':
        return [46.825905, -100.778275]
    elif state == 'Ohio':
        return [39.983334, -82.983330]
    elif state == 'South Dakota':
        return [44.3675, -100.3352]
    elif state == 'Wisconsin':
        return [43.073051, -89.401230]


# Function to update the map based on the selected year and energy type
def update_map(change):
    print(change)
#     selected_year = year_dropdown.value
    selected_energy = energy_dropdown.value
    add_markers_for_year_and_energy(selected_year, selected_energy)

# Create a dropdown widget for selecting energy types
energy_dropdown = Dropdown(options=df.columns[1:], value=df.columns[1], description='Energy Type:')
display(energy_dropdown)

# Connect the function to the dropdowns' value change events
# year_dropdown.observe(update_map, names='value')
energy_dropdown.observe(update_map, names='value')

# Create a dropdown widget for selecting years
# year_dropdown = Dropdown(options=df.columns, value=df.columns.min(), description='Year:')
# display(year_dropdown)

# Add a ClickForMarker feature to the map
click_for_marker = folium.ClickForMarker(popup='Click for marker')
click_for_marker.add_to(midwest_map)

# Display the map
midwest_map


Dropdown(description='Energy Type:', options=('Biomass_2001', 'Biomass_2002', 'Biomass_2003', 'Biomass_2004', …

{'name': 'value', 'old': 'Biomass_2001', 'new': 'Biomass_2002', 'owner': Dropdown(description='Energy Type:', index=1, options=('Biomass_2001', 'Biomass_2002', 'Biomass_2003', 'Biomass_2004', 'Biomass_2005', 'Biomass_2006', 'Biomass_2007', 'Biomass_2008', 'Biomass_2009', 'Biomass_2010', 'Biomass_2011', 'Biomass_2012', 'Biomass_2013', 'Biomass_2014', 'Biomass_2015', 'Biomass_2016', 'Biomass_2017', 'Biomass_2018', 'Biomass_2019', 'Biomass_2020', 'Biomass_2021', 'Geothermal_2000', 'Geothermal_2001', 'Geothermal_2002', 'Geothermal_2003', 'Geothermal_2004', 'Geothermal_2005', 'Geothermal_2006', 'Geothermal_2007', 'Geothermal_2008', 'Geothermal_2009', 'Geothermal_2010', 'Geothermal_2011', 'Geothermal_2012', 'Geothermal_2013', 'Geothermal_2014', 'Geothermal_2015', 'Geothermal_2016', 'Geothermal_2017', 'Geothermal_2018', 'Geothermal_2019', 'Geothermal_2020', 'Geothermal_2021', 'Hydropower_2000', 'Hydropower_2001', 'Hydropower_2002', 'Hydropower_2003', 'Hydropower_2004', 'Hydropower_2005', 'Hyd

In [ ]:
kw = {"location": [48, -102], "zoom_start": 3}

m = folium.Map(**kw)

states = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()
    

def style_function(feature):
    return {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 2
    }

def highlight_function(feature):
    return {'weight': 3, 'color': 'blue'}

folium.GeoJson(
    states,
    name="US States",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['State:']),
    highlight_function=highlight_function,
    popup=folium.GeoJsonPopup(fields=['name'], aliases=['State Name:'])
).add_to(m)

m